# Store sales predictions 

You are being asked to predict the sales of a bunch of stores, during the next 3 days.

You have a train and test set, as in the hackathon, and don't know the labels in the test set. 

The success metric is you small your mean absolute error is.

### General advice: 

- Get to a base result fast and early 


- Keep your functions well organized 


- Remember to sample intelligently. You are unlikely to want to train on all data every time 


- When dealing with a multi-index with lots of timeseries, explore a few individual ones to get an idea of the data


- Rolling train-test-predict may be tempting and elegant, but realistically you are unlikely to ever have enough CPU. 


- Use Index Slices to ensure legibility. You'll thank yourself later. 


- Don't go straight for the more sophisticated answers. Build up from the simper ones.


- Remember the business scope. What are we being asked to optimize here? 

In [ ]:
# usual imports 
import pandas as pd 
from matplotlib import pyplot as plt 
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm 
import warnings
warnings.filterwarnings(action="ignore")
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
plt.rcParams['figure.figsize'] = (16, 8)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  
% matplotlib inline 

from tqdm import tqdm_notebook

Let's get some data

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

-----

## Super basic understanding of the data 

How many rows and columns? 

In [ ]:
train.shape

What does it look like?

In [ ]:
train.head()

----

## Basic transformations 

#### Making it a datetime multi index 

This is clearly per-store and per-date, so let's make it a multi index. 

First, make the date a datetime: 

In [ ]:
train.Date = pd.to_datetime(train.Date, format='%Y-%m-%d')

What is the range of dates? 

In [ ]:
print(train.Date.min())
print(train.Date.max())

Great, let's set the index then: 

In [ ]:
train = train.set_index(['Date', 'Store'])

Always, always, always sort your index, or face a world of pain: 

In [ ]:
train = train.sort_index()

In [ ]:
train.head()

-----

## Exploring: single store analysis 

Let's observe a single store: 

In [ ]:
# remember index slicing? 
idx = pd.IndexSlice
store_1 = train.loc[idx[:, 1], :]

What does it look like? 

In [ ]:
store_1.head(3)

Let's see the store's sales 

In [ ]:
store_1.reset_index().set_index('Date').Sales.plot(figsize=(16, 4));

Our store seems to close on certain days. Maybe these are weekends? What's the mean sales per day?

In [ ]:
# get level values is a good way of getting the date values from the index, but there are others. 

store_1.groupby(store_1.index.get_level_values('Date').weekday_name).Sales.mean()

Got it, so Sundays are no sales days. 

-----

## Train test split 

Let's break off a few days to predict as a test set, and see if we can do it: 

1) We get the biggest timestamp in our index (i.e. the "maximum" date)

In [ ]:
last_day_in_index = train.index.get_level_values('Date').max()
print(last_day_in_index)

2) We want to predict only 4 days. This means that those days won't be in the training set.  To be the dates where the training set ends (`new_last_day_train`) and where the test starts (`new_first_day_test`)

In [ ]:
# let's peel off 4 days 
new_last_day_train = last_day_in_index - pd.DateOffset(days=4)
new_first_day_test = last_day_in_index - pd.DateOffset(days=3)

print('new_last_day_train', new_last_day_train)
print('new_first_day_test', new_first_day_test)

3) Finally, we split all time series at once using the index slicer

In [ ]:
# make new train and new test 
new_train = train.loc[idx[:new_last_day_train, :], :]
new_test = train.loc[idx[new_first_day_test:, :], :]

In [ ]:
new_train.tail(3)

In [ ]:
new_test.head(3)

To check if the test set is really only 4 days

In [ ]:
new_test.index.get_level_values('Date').unique()

Now, we have our train test split!

------

## Establish a baseline early 

Alright, now for a stupid question: how well would predict if we just predicted the last day's sale of each store, for every one of the next 4 days? 

In [ ]:
last_day = new_train.loc[idx[new_last_day_train]]

Here is our dumbass prediction: 

In [ ]:
last_day.Sales.head()

Ok, time to predict stuff. 

#### Day 1 of test set:

In [ ]:
new_test['predictions'] = 0 
new_test.head(5)

In [ ]:
days_in_test_set = new_test.index.get_level_values('Date').unique()
days_in_test_set

In [ ]:
for day in days_in_test_set: 
    new_test.loc[idx[day, :], 'predictions'] = last_day.Sales.values

To measure how well our approaches to modeling/prediction are, let's use [Mean Absolute Error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html). It is easy to interpret and we have the function implemented in sklearn. 

In [ ]:
new_test.sample(5)

In [ ]:
mean_absolute_error(y_true=new_test['Sales'], y_pred=new_test['predictions'])

Maybe we were unlucky, and got a Sunday in our test set? 

In [ ]:
days_in_test_set.weekday_name

Blast! A Sunday! 

Let's set the Sunday predictions to Zero, and see where that puts us. 

In [ ]:
new_test.loc[new_test.index.get_level_values('Date').weekday_name=='Sunday', 'predictions'] = 0

In [ ]:
mean_absolute_error(y_true=new_test['Sales'], y_pred=new_test['predictions'])

Interesting. To get an idea, what is the order of manitude of our sales? 

In [ ]:
new_train.Sales.hist(bins=20, figsize=(16, 4))

When the sales aren't 0, what's the median sale for one of our stores? 

In [ ]:
new_train.Sales.loc[new_train.Sales > 0].median()

So we're having an error which is approximately 1400, on sales whic are approximately 6500.

Now let's beat our stupid baseline, which is `1377`! 

-----

## Build on the previous baseline, with simple ideas 

For our second attempt, we'll do the following: take the previous sale, of each store, on the same weekday. 

Let's say we wanted to see the Fridays in our training set:

In [ ]:
fridays = new_train.loc[new_train.index.get_level_values('Date').weekday_name == 'Friday']

In [ ]:
fridays.tail()

Maybe stores are relatively consistent accross days of the week? 

In [ ]:
fridays.loc[idx[:, [1, 2, 3]], 'Sales'].unstack().plot(figsize=(16, 6));

Great. How do I get the most recent "Friday"? 

In [ ]:
most_recent = fridays.index.get_level_values('Date').max()

Well, they seem to have had some big slumps along the way, but other than that, fairly stable. Let's try this! 

In [ ]:
# let's just reset the predictions from our little test set... 
new_test['predictions'] = 0 

In [ ]:
for day in days_in_test_set:
    day_of_week = day.weekday_name
    same_day_of_the_week_data = new_train.loc[new_train.index.get_level_values('Date').weekday_name == day_of_week]
    date_of_most_recent = same_day_of_the_week_data.index.get_level_values('Date').max()
    
    new_test.loc[idx[day, :], 'predictions'] = new_train.loc[idx[date_of_most_recent, :], 'Sales'].values

In [ ]:
new_test.sample(5)

Any wins? 

In [ ]:
mean_absolute_error(y_true=new_test['Sales'], y_pred=new_test['predictions'])

Yes! 

------

## Finally, get your timeseries modelling face on! 

An important caveat. The techniques we taught you with ARIMA and SARIMAX are interesting to know, but in reality common sense is your main tool. Having made that caveat, let us proceed. 

Let's decompose a single store, to see what we can find 

In [ ]:
# matplotlib image size 
plt.rcParams['figure.figsize'] = (16, 8)

In [ ]:
# select a store 
a_store = new_train.loc[idx[:, 4], 'Sales']

# remove the level of the index that just says "4, 4, 4, 4 ..." because it's always the same store

a_store.index = a_store.index.droplevel('Store')

In [ ]:
decomposition = seasonal_decompose(a_store, model='additive')
decomposition.plot()
plt.show()

Now, about that seasonality... we kind of know it's weekly, so we should be able to guess we're gonna see some signal at 7 lags. Nevertheless, let's check the acf: 

In [ ]:
# matplotlib image size 
plt.rcParams['figure.figsize'] = (16, 4)

In [ ]:
plot_pacf(a_store, alpha=.05, lags=15)
plt.xlabel('lag')
plt.ylabel('Autocorrelation')
plt.show()

In [ ]:
plot_acf(a_store, alpha=.05, lags=15)
plt.xlabel('lag')
plt.ylabel('Autocorrelation')
plt.show()

Interesting... signal at 10 that does not seem exaplainable. Ah, the joys of Time Series! 

### _Igor, fetch me the SARIMAX..._
So extremely seasonal, as would be expected. Let's do some quick experiments with some hyper params, and try to fit a SARIMAX: 

Let's start with some dumb hyper params: 

In [ ]:
model = sm.tsa.statespace.SARIMAX(a_store,             
                          order=(0, 1, 0),              
                          seasonal_order=(1, 1, 1, 1), 
                          enforce_stationarity=False,  
                          enforce_invertibility=False) 

results = model.fit()
results.aic

Let's add in the fact that we suspect there is some seasonality, and S should be 7: 

In [ ]:
model = sm.tsa.statespace.SARIMAX(a_store,             
                          order=(0, 1, 0),              
                          seasonal_order=(1, 1, 1, 7), # <--- 7 
                          enforce_stationarity=False,  
                          enforce_invertibility=False) 

results = model.fit()
results.aic

Now, what should our `p` and `q` be, really?

Reminder: 
> For p use the PACF  
> For q use the ACF  

In [ ]:
model = sm.tsa.statespace.SARIMAX(a_store,             
                          order=(1, 1, 1),             # <---- 1, 1, 1    
                          seasonal_order=(1, 1, 1, 7), # <--- 7 
                          enforce_stationarity=False,  
                          enforce_invertibility=False) 

results = model.fit()
results.aic

Maybe S should go up to 10? 

In [ ]:
model = sm.tsa.statespace.SARIMAX(a_store,             
                          order=(1, 1, 1),             # <---- 1, 1, 1    
                          seasonal_order=(1, 1, 1, 10), # <--- 7 
                          enforce_stationarity=False,  
                          enforce_invertibility=False) 

results = model.fit()
results.aic

# Once you've covered obvious options, try using hyper parameter optimization 

Nop, back to 7. Let's do a basic hyper parameter optimizer 

_Note: if your computer is pre-2015, you might want to use someone else's for this bit, it's kind of heavy_

In [ ]:
import itertools

In [ ]:
# if you are feeling brave, increase the range to (0,3), 
# but it will take around 10 minutes on a good computer 

p = d = q = P = D = Q = range(0, 2)
S = 7

params_combinations = list(itertools.product(p, d, q, P, D, Q))

inputs = [[x[0], x[1], x[2], x[3], x[4], x[5], S] for x in params_combinations]

Note: in one of the following functions (`get_best_params`), we will use a really nice package: `tqdm`. It allows you to show progress bars in python consoles and notebooks

In [ ]:
def get_aic(series_, params):
    p = params[0] 
    d = params[1] 
    q = params[2] 
    P = params[3]
    D = params[4] 
    Q = params[5]
    S = params[6]
    
    model = sm.tsa.statespace.SARIMAX(series_,
                                      order=(p, d, q),
                                      seasonal_order=(P, D, Q, S),
                                      enforce_stationarity=False,
                                      enforce_invertibility=False)
    results = model.fit()
    
    return results.aic


def get_best_params(series_, inputs):
    
    aic_scores = {}
    params_index = {}
    for i in tqdm_notebook(range(len(inputs))):
        try: 
            param_set = inputs[i]
            aic = get_aic(series_, param_set) 
            aic_scores[i] = aic
            params_index[i] = param_set

        except Exception as e: 
            continue

    temp = pd.DataFrame(params_index).T
    temp.columns = ['p', 'd', 'q', 'P', 'D', 'Q', 'S']
    temp['aic'] = pd.Series(aic_scores)
    temp.sort_values('aic').head()

    best_model_params = temp.aic.idxmin()

    return temp.loc[best_model_params]

In [ ]:
%%time 
best_params = get_best_params(a_store, inputs)

In [ ]:
best_params

#### Now let's create a model with the winning params: 

Note: _The results I put in the model aren't actually the ones that are in the HPO, because we ran an even bigger search and found these to be even better. However, unless you have time to go for a walk, don't try to go for range(3) on all params_

In [ ]:
model = sm.tsa.statespace.SARIMAX(a_store,             
                          order=(1, 0, 2),             
                          seasonal_order=(1, 2, 2, 7),
                          enforce_stationarity=False,  
                          enforce_invertibility=False) 

results = model.fit()
results.aic

-----

## Take a look at the model in action: 

In [ ]:
def plot_predictions(series_, pred_):
    
    """ 
    Remember Sam told us to build functions as we go? Let's not write this stuff again. 
    """
    
    mean_predictions_ = pred_.predicted_mean

    pred_ci_ = pred_.conf_int()
    
    series_.plot(label='observed')
    mean_predictions_.plot(label='predicted', 
                           alpha=.7)

    plt.fill_between(pred_ci_.index,
                     pred_ci_.iloc[:, 0],
                     pred_ci_.iloc[:, 1], 
                     color='k', 
                     alpha=.2)
    plt.legend()
    plt.show()

In [ ]:
predictions = results.get_prediction(dynamic=False)
forecast = results.get_forecast(steps=4)

In [ ]:
plot_predictions(a_store, predictions)

In [ ]:
plot_predictions(a_store, forecast)

Hmm... not particularly sure this beats our hand-crafted predictions. Still, let's try it out! 

----

## Predicting all stores 

In [ ]:
def predict_with_sarimax(df_, store_nr, n_steps): 
    
    store_ = df_.loc[idx[:, store_nr], 'Sales']
    
    store_.index = store_.index.droplevel('Store')

    model = sm.tsa.statespace.SARIMAX(store_,             
                              order=(1, 0, 1),             
                              seasonal_order=(1, 1, 1, 7),
                              enforce_stationarity=False,  
                              enforce_invertibility=False) 

    results = model.fit()
    return results.get_forecast(steps=n_steps).predicted_mean

What stores do we have? 

In [ ]:
stores = train.index.get_level_values('Store').unique()

#### WARNING! If you have a slow computer, this might take a really, really long time. 
On a 2018 macbook pro, it takes about 60 seconds. 

In [ ]:
%%time 
# This is another cell that will take a long time to run. 

res = {}

for store_nr in tqdm_notebook(stores):
    res[store_nr] = predict_with_sarimax(df_=new_train, store_nr=store_nr, n_steps=4)

In [ ]:
results = pd.DataFrame(res).unstack().reset_index()
results.columns = ['Store', 'Date', 'Sales']
results = results.set_index(['Date', 'Store']).sort_index()

What do our results look like? 

In [ ]:
results.head(3)

In [ ]:
results.tail(3)

----

# Optional: using Joblib to speed up calculations 

Depending on what computer you have, the previous cell can take a loooong time. When our code has cycles where each iteration does not need any information from previous iterations, we can use a **REALLY** cool package called [joblib](https://pythonhosted.org/joblib/), a package to make distributed computing really easy

In [ ]:
from joblib import Parallel, delayed

In [ ]:
%%time 
# This is another cell that will take a long time to run. 

def wrap_predict_with_sarimax(df_, store_nr): 
    return (store_nr, predict_with_sarimax(df_, store_nr, n_steps=4))

res = Parallel(n_jobs=-1)(delayed(wrap_predict_with_sarimax)(df_=new_train, store_nr=store_nr) 
                          for store_nr in tqdm_notebook(stores))

res = dict(res)

In [ ]:
results = pd.DataFrame(res).unstack().reset_index()
results.columns = ['Store', 'Date', 'Sales']
results = results.set_index(['Date', 'Store']).sort_index()

In [ ]:
results.head(3)

In [ ]:
results.tail(3)

Same results but 

**waaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaay**

faster :D

------

## Let's see how we did! 

In [ ]:
for day in results.index.get_level_values('Date').unique():
    print(day)

In [ ]:
for day in days_in_test_set:
    new_test.loc[idx[day, :], 'predictions'] = results.loc[idx[day, :], 'Sales'].values

In [ ]:
new_test.head()

In [ ]:
mean_absolute_error(y_true=new_test['Sales'], y_pred=new_test['predictions'])

Wow... alright, that's kind of impressive. And we haven't added exogenous yet :)

Also, we only tuned on one store, maybe we can do better? 

----

## Predicting the test set 

By now, Sam is screaming **_"30 minutes to deliver!!"_**. Panic fills the room. Let's predict the test set. 

##### Prep the test set 
_(in your heart of hearts, you know you should have made functions. But not Sam is screaming, so no time for that anymore)_

In [ ]:
test.Date = pd.to_datetime(test.Date, format='%Y-%m-%d')

In [ ]:
test = test.set_index(['Date', 'Store'])

In [ ]:
test = test.sort_index()  # <--- seriously. 

In [ ]:
test.head()

#### Make predictions 

Get the training data instead of the "new train": 

In [ ]:
train.head(3)

How many days are we supposed to predict in the test set? 

In [ ]:
days_in_test = test.index.get_level_values('Date').unique()
sorted(days_in_test)

Ok, so we need to forecast 4 days, from the full training set. We've pretty much done this before! 

In [ ]:
%%time 
# This is another cell that will take a long time to run. 

res = {}

for store_nr in tqdm_notebook(stores):
    res[store_nr] = predict_with_sarimax(df_=train, store_nr=store_nr, n_steps=4)

In [ ]:
results = pd.DataFrame(res).unstack().reset_index()
results.columns = ['Store', 'Date', 'Sales']
results = results.set_index(['Date', 'Store']).sort_index()

In [ ]:
for day in days_in_test:
    test.loc[idx[day, :], 'Sales'] = results.loc[idx[day, :], 'Sales'].values

In [ ]:
test.head()

-------

# Delivered! Time for some closing remarks.

You won't need this section to pass your BLU3 exercises, however it contains good advice for the Hackathon. 

----

## Choosing your metric 

We chose Mean Absolute Error, but depending on the problem, different metrics are preferable. 

### Is there a better metric? 

Imagine that we have the following targets and predictions

In [ ]:
df = pd.DataFrame({
    'prediction': [970, 10, 15, 10000, 30, 90, 4700], 
    'target': [1000, 20, 5, 9000, 35, 70, 5001]
})

df

if we use MAE, the error for the line 3 will be higher than the one we get for line 5

In [ ]:
df['abs. error'] = abs(df['prediction'] - df['target'])

df

But does that mean that missing by 10 in line 2 is less important than missing by 1000 in line 3? Imagine that line 2 is the prediction and target for a really expensive product while line 3 is a cheap product. One way to weight the error would be to use the `sample_weight` parameter that sklearn offers in many of its metrics. But that would require us to decide what weights we would use for each error. An alternative to MAE that makes that evaluates the weight of each error is the [Mean Absolute Percentage Error (MAPE)](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)

In [ ]:
df['abs. perc. error'] = abs((df['prediction'] - df['target']) / df['target'])

df

with MAPE, the error for line 3 is not as important as the one for line 2. For MAPE, we relate the error unit to the desired target. The smaller the desired target is, the higher the impact of the error. We measured the errors for each line but we still need to compute the mean for each one

In [ ]:
df[['abs. error', 'abs. perc. error']].mean()

Oh, by the way, the dataset we used in this notebook was included in a [Kaggle challenge](https://www.kaggle.com/c/rossmann-store-sales) where the evaluation metric was [Root Mean Square Percentage Error](https://www.kaggle.com/c/rossmann-store-sales#evaluation). This metric is to RMSE as MAE is to MAPE.

----

## Performing better train test splits 

### Are there other ways of performing train-test split for time series? 

Unlike the datasets we used in the bootcamp and first hackathon where we wouldn't have any issues if we performed random train-test splits, in time series we must respect the order between train and test. Simply put, **all timestamps that appear in the train set must be smaller than the ones in test set**. A simple way to perform the train-test split in time series is to choose an arbitrary date like we did and split the time series into train and test. But, if we do this, we aren't able to check how well our model generalizes to different parts of the time series. An alternative train-test split method is the one implemented in sklearn, [TimeSeriesSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html). Before using it, let's extract the time series for store 1

In [ ]:
all_data = pd.concat((train, test))
all_data_for_store_1 = all_data.loc[idx[:, 1], :]
all_data_for_store_1 = all_data_for_store_1.reset_index().drop('Store', axis=1).set_index('Date').sort_index()

Now, let's use `TimeSeriesSplit`

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
tss = TimeSeriesSplit(3, max_train_size=70)

i = 1

for endo_train_idx, endo_test_idx in tss.split(all_data_for_store_1):
    pd.Series(i, index=endo_train_idx).plot(label="Split {} (train)".format(i))
    pd.Series(i, index=endo_test_idx).plot(label="Split {} (test)".format(i))
    print("SPLIT {}".format(i))
    print('\ttrain idx', '({} observations)'.format(len(endo_train_idx)), '\n', endo_train_idx)
    print('\ttest idx', '({} observations)'.format(len(endo_test_idx)), '\n', endo_test_idx)
    i += 1
    print('\n')
    
plt.legend();

So, essentially, `TimeSeriesSplit` divides the dataset into *K* overlapping parts, where each one of those is divided into train and test. If you set `max_train_size` to an integer, the splitting algorithm will create splits where the train set has, at most, `max_train_size` observations. If you set `max_train_size=None`

In [ ]:
tss = TimeSeriesSplit(3, max_train_size=None)

i = 1

for endo_train_idx, endo_test_idx in tss.split(all_data_for_store_1):
    pd.Series(i, index=endo_train_idx).plot(label="Split {} (train)".format(i))
    pd.Series(i, index=endo_test_idx).plot(label="Split {} (test)".format(i))
    print("SPLIT {}".format(i))
    print('\ttrain idx', '({} observations)'.format(len(endo_train_idx)), '\n', endo_train_idx)
    print('\ttest idx', '({} observations)'.format(len(endo_test_idx)), '\n', endo_test_idx)
    i += 1
    print('\n')
    
plt.legend();

the train set will always start at observation 0 but the its size will expand for each new split. Both of them can be used as a way to implement time series cross validation.